### Practice: Parameter Efficient Fine-Tuning
In this notebook, you're gonna fine-tune large language models within limited GPU memory.

In [1]:
#%pip install --quiet transformers==4.34.1 accelerate==0.24.0 sentencepiece==0.1.99 optimum==1.13.2 peft==0.5.0 bitsandbytes==0.41.2.post2

import torch
import torch.nn as nn
import torch.nn.functional as F

import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
model_name = 'Enoch/llama-7b-hf'

# загрузка токенизатора Llama ...
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

# ... и самой модели
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # веса модели — 4-битные; нормировка по слоям и активации — в формате fp32
)
for param in model.parameters():
    param.requires_grad=False

model.gradient_checkpointing_enable()  # Включает градиентный чекпоинтинг, который позволяет экономить память, сохраняя лишь малую часть активаций модели.
                                       # Во время обратного прохода (backward pass) остальные активации пересчитываются заново, что уменьшает использование оперативной памяти,
                                       # но увеличивает вычислительные затраты, так как некоторые слои модели повторно вычисляются. Полезно для обучения на устройствах с ограниченной памятью.

model.enable_input_require_grads()     # Переопределяет настройку, которая по умолчанию отключает градиенты для входных данных в режиме градиентного чекпоинтинга.
                                       # Эта функция необходима, если мы хотим вычислять производные по входным данным, позволяя входам модели требовать градиенты.
# больше о градиентном чекпоинтинге: https://pytorch.org/docs/stable/checkpoint.html https://arxiv.org/abs/1604.06174

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

### Prompt tuning: the story of a fox (1 балл)

![img](https://i.imgur.com/Ux3qQAu.png) (source: theodd1souts.fandom.com)

In [3]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

for i in range(10):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))


Output: <s>A quick brown fox jumps over the lazy dog.
A quick


Какое вопиющее вранье! Этот самый лис уверяет вас, что он, вообще-то, не перепрыгивал через ленивую собаку. Нет, сэр! Лис просто занимался своими делами. Ваша задача — обучить модель говорить правду: сегодня через собаку никто не прыгал.

In [4]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)
outputs = model(**batch)

next_word_logits = outputs.logits[:, :-1]
true_next_tokens = batch['input_ids'][:, 1:]
loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))

print("Loss:", loss)

Loss: tensor(3.0729, device='cuda:0', grad_fn=<NllLossBackward0>)


Except, we can't train the entire model - that would be 28GB gradients in float32. Instead, let's run [prompt tuning](https://arxiv.org/abs/2104.08691).

![img](https://i.imgur.com/VwNNKnb.png)


In [5]:
class WordEmbeddingsWithLearnedPrompts(nn.Module):
    """
    Для настройки prompt-тюнинга вам нужно заменить оригинальные эмбеддинги слов модели на слой — ЭТОТ слой, —
    который вставляет обучаемые промпты вместо первых N эмбеддингов токенов.
    """

    def __init__(self, word_embeddings: nn.Embedding, num_prompts: int):
        super().__init__()
        self.original_word_embeddings = word_embeddings
        self.num_prompts = num_prompts
        self.learnable_prompts = nn.Parameter(
            torch.randn(1, num_prompts, word_embeddings.embedding_dim), requires_grad=True)

    def forward(self, input_ids: torch.LongTensor):
        # форма input_ids: [размер батча, длина последовательности]
        assert input_ids.dtype == torch.int64
        assert input_ids.shape[1] > self.num_prompts
        assert torch.all(input_ids[:, :self.num_prompts] == tokenizer.pad_token_id).item(), "не забудьте добавить несколько токенов BOS в начало input_ids"

        # Ваша задача: создать эмбеддинги для input_ids, но заменить первые :num_prompts: токены на self.learnable_prompts
        # Это связано с тем, что мы добавим :num_prompts: токенов-паддингов в начале

        # После выполнения вы должны получить векторное представление для каждого токена в input_ids,
        # но первые :num_prompts: векторов должны быть равны learnable_prompts;
        # остальные векторы после первых :num_prompts: должны быть обычными эмбеддингами
        # Замечание: поскольку вы работаете с обучаемыми параметрами, используйте torch.cat вместо присваивания элементов

        # Код для вставки промптов:
        embed = self.original_word_embeddings(input_ids)
        learnable = self.learnable_prompts.expand(input_ids.size(0), -1, -1)
        old_embed = embed[:, self.num_prompts:, :]
        embedded_inputs_with_prompts = torch.cat([learnable, old_embed], dim=1)

        return embedded_inputs_with_prompts

In [6]:
num_prompts = 16
test_emb_layer = WordEmbeddingsWithLearnedPrompts(model.model.embed_tokens, num_prompts=num_prompts).to(device)
test_input_ids = tokenizer("a cat say on a may", return_tensors='pt')['input_ids'].to(device)

space_for_prompts = torch.full([len(test_input_ids), num_prompts], fill_value=tokenizer.pad_token_id,
                               dtype=torch.int64, device=device)
test_inputs_with_prompts = torch.cat([space_for_prompts, test_input_ids], dim=1)

with torch.cuda.amp.autocast():
  test_prompt_embeddings = test_emb_layer(test_inputs_with_prompts)

assert test_prompt_embeddings.shape[:2] == test_inputs_with_prompts.shape
assert test_prompt_embeddings.shape[-1] == model.config.hidden_size
assert torch.allclose(test_prompt_embeddings[:, :num_prompts], test_emb_layer.learnable_prompts.float())
assert torch.allclose(test_prompt_embeddings[:, num_prompts:], model.model.embed_tokens(test_input_ids).float())
print("Looks legit!")

Looks legit!


/tmp/ipykernel_15983/1152930240.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


__Now that it works,__ let's inject learnable prompts into the main model and teach it about foxes.

In [7]:
assert isinstance(model.model.embed_tokens, nn.Embedding), "you have already replaced the embedding layer. If the replacement is broken, please reload the model"

model.model.embed_tokens = WordEmbeddingsWithLearnedPrompts(model.model.embed_tokens, num_prompts=num_prompts).to(device)

opt = torch.optim.Adam([model.model.embed_tokens.learnable_prompts], lr=0.01)

In [9]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False).to(device)
space_for_prompts = torch.full([len(test_input_ids), num_prompts], fill_value=tokenizer.pad_token_id,
                               dtype=torch.int64, device=device)

batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)
outputs = model(**batch)
next_word_logits = outputs.logits[:, num_prompts : -1, :]
true_next_tokens = batch['input_ids'][:, num_prompts + 1:]
loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
print("Loss:", loss)

Loss: tensor(8.1688, device='cuda:0', grad_fn=<NllLossBackward0>)


In [10]:
for epoch in tqdm(range(100)):
  opt.zero_grad()
  outputs = model(**batch)
  next_word_logits = outputs.logits[:, num_prompts : -1, :]
  true_next_tokens = batch['input_ids'][:, num_prompts + 1:]
  loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))
  loss.backward()
  opt.step()

  0%|          | 0/100 [00:00<?, ?it/s]

In [11]:
assert loss.item() <= 0.1
print("Good job!")

Good job!


In [12]:
prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
batch['input_ids'] = torch.cat([space_for_prompts, batch['input_ids']], dim=1)
batch['attention_mask'] = torch.cat([torch.ones_like(space_for_prompts), batch['attention_mask']], dim=1)


for i in range(15):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0, num_prompts:].cpu().numpy().tolist()))

# if you did everything right, the model will deny that the fox jumped over the lazy dog


Output: <s>A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it


### Using HuggingFace PEFT (1 балла)

[`peft`](https://huggingface.co/docs/peft/index) — это библиотека, связанная с Transformers от Hugging Face, которая позволяет применять различные методы __p__arameter __e__fficient __f__ine-__t__uning (peft) к предварительно обученным трансформерам. Библиотека реализует и prompt tuning, и prefix tuning и несколько методов на основе адаптеров в едином интерфейсе.




In [14]:
model_name = 'Enoch/llama-7b-hf'

# загрузка токенизатора Llama ...
tokenizer = transformers.LlamaTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id

# ... и самой модели
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # веса модели — 4-битные; нормировка по слоям и активации — в формате fp32
)
for param in model.parameters():
    param.requires_grad=False

model.gradient_checkpointing_enable()  # Включает градиентный чекпоинтинг, который позволяет экономить память, сохраняя лишь малую часть активаций модели.
                                       # Во время обратного прохода (backward pass) остальные активации пересчитываются заново, что уменьшает использование оперативной памяти,
                                       # но увеличивает вычислительные затраты, так как некоторые слои модели повторно вычисляются. Полезно для обучения на устройствах с ограниченной памятью.

model.enable_input_require_grads()     # Переопределяет настройку, которая по умолчанию отключает градиенты для входных данных в режиме градиентного чекпоинтинга.
                                       # Эта функция необходима, если мы хотим вычислять производные по входным данным, позволяя входам модели требовать градиенты.
# больше о градиентном чекпоинтинге: https://pytorch.org/docs/stable/checkpoint.html https://arxiv.org/abs/1604.06174

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [15]:
import peft
assert isinstance(model.model.embed_tokens, nn.Embedding), "please reload the model"

peft_config = peft.PromptTuningConfig(task_type=peft.TaskType.CAUSAL_LM, num_virtual_tokens=16)
model = peft.get_peft_model(model, peft_config)  # note: for most peft methods, this line also modifies model in-place
print("Trainable parameters:", sum(p.numel() for p in model.parameters() if p.requires_grad))
print("Total parameters (excluding quantization):", sum(p.numel() for p in model.parameters()))

Trainable parameters: 65536
Total parameters (excluding quantization): 3500478464


In [ ]:
# Your task: optimize the PEFT-wrapped model to achieve next token prediction loss < 0.1, but this time using PEFT
# Please note: you no longer need to prepend PAD tokens, but you still need to skip :num_virtual_tokens: first logits.
# Finally, generate the sentence to make sure that the model learned the truth.Задание: оптимизировать модель с использованием PEFT для достижения потерь на предсказание следующего токена < 0.1, но на этот раз использовать PEFT.

In [16]:
the_truth = "A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it anyway!"
batch = tokenizer(the_truth, return_tensors='pt', return_token_type_ids=False)
batch = {k: v.to(device) for k, v in batch.items()}
opt = torch.optim.Adam(model.prompt_encoder.parameters(), lr=0.01)

In [17]:
for i in tqdm(range(100)):
    outputs = model(**batch)

    next_word_logits = outputs.logits[:, num_prompts : -1, :]
    true_next_tokens = batch['input_ids'][:, 1:]
    loss = F.cross_entropy(next_word_logits.flatten(0, 1), true_next_tokens.flatten(0, 1))

    opt.zero_grad()
    loss.backward(retain_graph=True)
    opt.step()

  0%|          | 0/100 [00:00<?, ?it/s]

In [19]:
print(f'loss: {loss.item():.4f}')
assert loss.item() <= 0.1
print("Good job!")

loss: 0.0050
Good job!


In [20]:
model.eval()

prompt = 'A quick brown fox'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)

for i in range(15):
    next_token = model(**batch).logits[0, -1].argmax(-1).reshape(1, 1)
    batch['input_ids'] = torch.cat([batch['input_ids'], next_token], dim=-1)
    batch['attention_mask'] = torch.cat([batch['attention_mask'], torch.ones_like(next_token)], dim=-1)

print("\nOutput:", tokenizer.decode(batch['input_ids'][0].cpu().numpy().tolist()))

# if you did everything right, the model will deny that the fox jumped over the lazy dog


Output: <s>A quick brown fox did not jump over the lazy dog. Besides, that dog deserved it


### PEFT with LoRA (1 балл)

При обучении на более сложных задачах можно использовать адаптеры низкого ранга на основе статьи о LoRA.

Основная идея заключается в добавлении адаптеров низкого ранга параллельно существующим линейным слоям, как показано здесь:

<center><img src="https://i.imgur.com/6bQLNiG.png" width=240px></center>


В оригинальной статье по LoRA адаптеры добавлялись только к матрицам attention. Однако, последующие работы показывают, что полезно адаптировать также FFN. Но прежде чем приступить к обучению, нам нужно реализовать базовый слой LoRA.

In [33]:
# re-load the model to remove any previous PEFT tuners
model = transformers.AutoModelForCausalLM.from_pretrained(
    #model_name, device_map='auto', low_cpu_mem_usage=True, offload_state_dict=True,
    model_name, low_cpu_mem_usage=True, offload_state_dict=True,
    load_in_4bit=True, torch_dtype=torch.float32,  # weights are 4-bit; layernorms and activations are fp32
)
for param in model.parameters():
    param.requires_grad=False
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacity of 15.58 GiB of which 42.00 MiB is free. Including non-PyTorch memory, this process has 14.99 GiB memory in use. Of the allocated memory 14.12 GiB is allocated by PyTorch, and 596.67 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [29]:
class LoRALayer(nn.Module):
    """Wraps a linear layer with LoRA-like adapter. Wraps an existing OPT linear layer"""
    def __init__(self, module: nn.Linear, rank: int):
        super().__init__()
        self.module = module  # pre-trained (frozen) linear layer
        self.adapter_A = nn.Parameter(torch.empty(module.in_features, rank, device=module.weight.device))
        nn.init.kaiming_uniform_(self.adapter_A, a=5 ** 0.5)
        self.adapter_B = nn.Parameter(torch.zeros(rank, module.out_features, device=module.weight.device))

    def forward(self, input):
        # Apply self.module and LoRA adapter, return the sum (self.module outputs + adapter outputs)
        module_outputs = self.module(input)
        temp = input @ self.adapter_A
        lora_output = temp @ self.adapter_B

        return module_outputs + lora_output

In [30]:
# test your implementation
test_linear = nn.Linear(128, 128)
test_linear.weight.data[...] = torch.eye(128)
test_adapter = LoRALayer(test_linear, rank=8)

assert torch.allclose(test_adapter(torch.ones(1, 1, 128)), test_linear.bias + 1), "please check your forward pass"

test_adapter.adapter_A.data[...] = torch.linspace(0.1, -0.5, 128 * 8).view(128, 8)
test_adapter.adapter_B.data[...] = torch.linspace(0.5, -0.1, 128 * 8).view(8, 128)
test_linear.bias.data[...] = torch.linspace(1., -1., 128)

dummy_loss = F.mse_loss(test_adapter(torch.ones(1, 128) / 128).squeeze(), torch.linspace(-1, 1, 128))
assert torch.allclose(dummy_loss, torch.tensor(1.3711389), rtol=0, atol=1e-4)
dummy_loss.backward()
assert all(w.grad is not None for w in [test_adapter.adapter_A, test_adapter.adapter_B]), "some adapter weights have no grad"
assert torch.allclose(test_adapter.adapter_A.grad.sum(), torch.tensor(-0.60158), rtol=0, atol=1e-4), "bad grad w.r.t. A"
assert torch.allclose(test_adapter.adapter_B.grad.sum(), torch.tensor(0.9931), rtol=0, atol=1e-4), "bad grad w.r.t. B"
# note: bad grad means that your code is different from LoRA paper OR that your code is not autograd-friendly (e.g. no_grad)
del dummy_loss, test_linear, test_adapter
print("All tests passed!")

All tests passed!


### Apply LoRA to the model

The code below applies LoRA adapters on top of Q/K/V linear layers in Llama attention. You may also choose to modify other layers:
* self_attn.o_proj - attention output projection
* mlp.up_proj, mlp.gate_proj, mlp.down_proj - transformer feedforward layers
* lm_head - output LM head

__Note:__ please scroll down for the homework task

In [32]:
lora_rank = 8

for name, module in model.model.layers.named_modules():
    if 'LlamaDecoderLayer' in repr(type(module)):
        module.self_attn.q_proj = LoRALayer(module.self_attn.q_proj, rank=lora_rank).to(device)
        module.self_attn.k_proj = LoRALayer(module.self_attn.k_proj, rank=lora_rank).to(device)
        module.self_attn.v_proj = LoRALayer(module.self_attn.v_proj, rank=lora_rank).to(device)

assert sum(isinstance(module, LoRALayer) for module in model.modules()) == 96  # for Llama-7B

AttributeError: 'LoRALayer' object has no attribute 'in_features'

In [25]:
batch = tokenizer("This model wants to share its greatest secret:", return_tensors='pt', return_token_type_ids=False)
# test a single training step, make sure we get meaningful gradients
with torch.cuda.amp.autocast(dtype=torch.float32):
    out = model.forward(**batch)
    (out.logits.norm() / 100).backward()

for i, module in enumerate(model.modules()):
    if isinstance(module, LoRALayer):
        assert module.adapter_B.grad is not None
        assert module.adapter_B.grad.norm().item() > 0

model.zero_grad(set_to_none=True)
print("Grad check successful, well done!")

/tmp/ipykernel_15983/3590305689.py:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(dtype=torch.float32):


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

### (example) How to train your model

Чисто пример как учить модельку

Там основная таска ниже

In [ ]:
import datasets
data = datasets.load_dataset("Abirate/english_quotes", split="train[:32]") # 32 lines
data = data.map(lambda samples: tokenizer(samples['quote']), batched=True)
model._hf_peft_config_loaded = True  # silence a warning from HF trainer

trainer = transformers.Trainer(
    model=model, train_dataset=data,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2, gradient_accumulation_steps=1,
        warmup_steps=250, max_steps=100, learning_rate=2e-4, fp16=True,
        logging_steps=1, output_dir='outputs', report_to=None),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()

# Это просто пример


### Финальная задача: *реально* обучить модель (3 балла)

Ваша задача — дообучить модель для _генерации кода на Python_. Более конкретно:

* __датасет:__ используйте [codeparrot-clean](https://huggingface.co/datasets/codeparrot/codeparrot-clean) или любой другой набор данных, содержащий Python-код. Так как для этой задачи не требуется много данных, будет достаточно использовать чистый `codeparrot`
* __короткие строки:__ возьмите первые 512 символов каждой строки
* __тип адаптера:__ используйте LoRA, как описано выше, __и, кроме того, хотя бы один из следующих элементов:__
   - дополнительный адаптер для lm_head
   - дополнительный адаптер для компонентов MLP (mlp.*)

* __обучение:__ вам не нужно обучать до сходимости. Если всё настроено правильно, модель должна начать `.generate` код уже после 500 шагов. Пожалуйста, используйте размер батча минимум 4 (4 x 1 x 512 токенов), при этом установите `gradient_accumulation_steps=4`.

Примечание: библиотека `peft` также имеет реализацию LoRA. Воспользуйтесь этим

__Альтернативное задание:__
Если не нравится такое задание - возьми другой датасет и дообучи на свою задачу. Заодно и напиши про результаты)
думаю, они будут очень приятны


In [ ]:
prompts =  ['', 'import', 'from', 'while', 'try', 'if', 'for', 'torch']  # добавьте ещё несколько, которые не обязательно ассоциируются с Python

# <ОЧЕНЬ МНОГО ВАШЕГО КОДА>
# нужно подготовить некоторые примеры (или "выборки") для проверки вашего fine-tuned модели
# можно использовать transformers.Trainer (как выше) или свой код для тренировки
# после завершения тренировки покажите примеры текста, сгенерированного вашей моделью, предполагается, что это будут фрагменты кода на Python
# красиво выведите примеры генерации (рекомендация: используйте pandas или HTML) для удобного сравнения
# примечание: ваша модель с улучшением через LoRA может генерировать текст так же, как и не обученная модель (как выше)

In [ ]:

from IPython.display import HTML, display
table_template = """<table style="border:1px solid black" >
  <tr>
    <th style="text-align: center; border:1px solid black">PROMPT</th>
    <th style="text-align: center; border:1px solid black">BEFORE</th>
    <th style="text-align: center; border:1px solid black">AFTER</th>
  </tr>
{}
</table>"""

row_template = '''  <tr>
    <td style="width:20%; border:1px solid black"><pre align="left">`{}`</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
    <td style="width:40%; border:1px solid black"><pre align="left">{}</pre></td>
  </tr>'''

rows = []

for prompt in prompts:
    # replace placeholders in the format() arguments
    rows.append(row_template.format(prompt, "BEFORE FINETUNING", "TO BE GENERATED AFTER FINETUNING"))

display(HTML(table_template.format('\n'.join(rows))))

PROMPT,BEFORE,AFTER
``,BEFORE FINETUNING,TO BE GENERATED AFTER FINETUNING
`import`,BEFORE FINETUNING,TO BE GENERATED AFTER FINETUNING
`from`,BEFORE FINETUNING,TO BE GENERATED AFTER FINETUNING
`while`,BEFORE FINETUNING,TO BE GENERATED AFTER FINETUNING
`try`,BEFORE FINETUNING,TO BE GENERATED AFTER FINETUNING
`if`,BEFORE FINETUNING,TO BE GENERATED AFTER FINETUNING
`for`,BEFORE FINETUNING,TO BE GENERATED AFTER FINETUNING
`torch`,BEFORE FINETUNING,TO BE GENERATED AFTER FINETUNING


По-моему крутой результат должен выйти )

опиши забавные инсайды, которые получил после выполнения этого задания